##Basic Idea

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
# Generate a small synthetic dataset
torch.manual_seed(42)  # Set seed for reproducibility

# Number of samples in the dataset
num_samples = 100

# Input features (random numbers)
X = torch.randn(num_samples, 5)  # 5 input features

# Target labels (0 for "no," 1 for "yes")
y = torch.randint(2, (num_samples,), dtype=torch.long)

# Create a DataLoader for the dataset
batch_size = 10
dataset = TensorDataset(X, y)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Define the neural network model
class CustomModel(nn.Module):
    def __init__(self, input_size, use_yes_neurons=True):
        super(CustomModel, self).__init__()
        self.use_yes_neurons = use_yes_neurons
        self.fc = nn.Linear(input_size, 2)  # 2 output neurons

    def forward(self, x):
        if self.use_yes_neurons:
            # When using "yes" neurons, set the output for "no" class to a large negative value
            return self.fc(x) + torch.tensor([0.0, -1000.0], device=x.device)
        else:
            # When using "no" neurons, set the output for "yes" class to a large negative value
            return self.fc(x) + torch.tensor([-1000.0, 0.0], device=x.device)

# Instantiate the model with the toggle set to "yes" initially
model = CustomModel(input_size=5, use_yes_neurons=True)

# Define the loss function (cross-entropy loss)
criterion = nn.CrossEntropyLoss()

# Define the optimizer (stochastic gradient descent)
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training loop
num_epochs = 50

for epoch in range(num_epochs):
    for inputs, labels in dataloader:
        # Forward pass
        outputs = model(inputs)

        # Compute the loss
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print the loss after each epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Now, you can use the trained model with the toggle set to "yes" or "no"
# For example, if toggle is set to "yes":
# model.use_yes_neurons = True
# If toggle is set to "no":
# model.use_yes_neurons = False

# Test the model with some example inputs
example_input = torch.randn(1, 5)  # 1 example with 5 input features
model.use_yes_neurons = True  # Set the toggle to "yes"
output_yes = model(example_input)
print(f'Toggle set to "yes": {output_yes}')

model.use_yes_neurons = False  # Set the toggle to "no"
output_no = model(example_input)
print(f'Toggle set to "no": {output_no}')


Epoch [1/50], Loss: 399.7482
Epoch [2/50], Loss: 699.6611
Epoch [3/50], Loss: 499.6779
Epoch [4/50], Loss: 499.8286
Epoch [5/50], Loss: 399.5229
Epoch [6/50], Loss: 399.3839
Epoch [7/50], Loss: 399.7384
Epoch [8/50], Loss: 699.0129
Epoch [9/50], Loss: 599.2856
Epoch [10/50], Loss: 798.9515
Epoch [11/50], Loss: 698.8080
Epoch [12/50], Loss: 598.7293
Epoch [13/50], Loss: 598.9779
Epoch [14/50], Loss: 698.5979
Epoch [15/50], Loss: 199.7749
Epoch [16/50], Loss: 698.1057
Epoch [17/50], Loss: 598.9513
Epoch [18/50], Loss: 698.3634
Epoch [19/50], Loss: 398.8497
Epoch [20/50], Loss: 897.4589
Epoch [21/50], Loss: 698.1982
Epoch [22/50], Loss: 598.1552
Epoch [23/50], Loss: 398.8141
Epoch [24/50], Loss: 398.6716
Epoch [25/50], Loss: 597.6886
Epoch [26/50], Loss: 598.0131
Epoch [27/50], Loss: 597.7574
Epoch [28/50], Loss: 398.2762
Epoch [29/50], Loss: 497.8211
Epoch [30/50], Loss: 597.3702
Epoch [31/50], Loss: 598.1916
Epoch [32/50], Loss: 597.0110
Epoch [33/50], Loss: 597.6887
Epoch [34/50], Loss

In [ ]:
import torch
import torch.nn as nn

# Define the neural network model
class CustomModel(nn.Module):
    def __init__(self, input_size, use_yes_neurons=True):
        super(CustomModel, self).__init__()
        self.use_yes_neurons = use_yes_neurons
        self.fc = nn.Linear(input_size, 2)  # 2 output neurons

    def forward(self, x):
        if self.use_yes_neurons:
            # When using "yes" neurons, set the output for "no" class to a large negative value
            return self.fc(x) + torch.tensor([0.0, -1000.0], device=x.device)
        else:
            # When using "no" neurons, set the output for "yes" class to a large negative value
            return self.fc(x) + torch.tensor([-1000.0, 0.0], device=x.device)

# Instantiate the model with the toggle set to "yes" initially
model = CustomModel(input_size=5, use_yes_neurons=True)

# Function to toggle between "yes" and "no" neurons
def toggle_neurons():
    model.use_yes_neurons = not model.use_yes_neurons
    print(f"Neurons set to {'yes' if model.use_yes_neurons else 'no'}")

# Chatbot interface
print("Chatbot Interface - Type 'T' to toggle neurons, 'exit' to quit.")
while True:
    user_input = input("User: ")

    if user_input.lower() == 'exit':
        break
    elif user_input.lower() == 't':
        toggle_neurons()
    else:
        # Process user input using the model
        input_tensor = torch.randn(1, 5)  # Random input for demonstration
        model_output = model(input_tensor)
        predicted_class = torch.argmax(model_output, dim=1).item()

        # Display chatbot response based on the model's prediction
        if predicted_class == 0:
            print("Chatbot: No")
        else:
            print("Chatbot: Yes")


##Basic Idea 2.0

In [1]:
# Install the transformers library
!pip install transformers

In [2]:
import torch
import json
from transformers import GPT2LMHeadModel, GPT2Config, GPT2Tokenizer

In [3]:
def convert_to_numpy(obj):
    if isinstance(obj, torch.Tensor):
        return obj.detach().cpu().numpy().tolist()
    elif isinstance(obj, dict):
        return {key: convert_to_numpy(value) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [convert_to_numpy(item) for item in obj]
    else:
        return obj

# Function to save model weights to a file
def save_model_weights(model, filename):
    torch.save(model.state_dict(), filename)

# Function to load model weights from a file
def load_model_weights(model, filename):
    model.load_state_dict(torch.load(filename))

In [ ]:
# Load pre-trained GPT-2 model
pretrained_model_name = "gpt2"
pretrained_model = GPT2LMHeadModel.from_pretrained(pretrained_model_name)

#loading pre-trained config to empty model
empty_config = GPT2Config.from_dict(pretrained_model.config.to_dict())

# Save the state_dict to a JSON file
save_model_weights(pretrained_model, "gpt2_weights.json")

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

In [ ]:
del pretrained_model

In [ ]:
# Load the state_dict from the JSON file into an empty GPT-2 model
empty_model = GPT2LMHeadModel(empty_config)

In [ ]:
import io
def load_model_weights_in_chunks(model, filename, chunk_size=1024 * 1024 * 50):  # Adjust the chunk size as needed
    with open(filename, "rb") as file:
        buffer = file.read(chunk_size)
        while buffer:
            state_dict_chunk = torch.load(io.BytesIO(buffer))
            model.load_state_dict(state_dict_chunk, strict=False)
            buffer = file.read(chunk_size)


In [ ]:
load_model_weights(empty_model, "gpt2_weights.json")

# Now, empty_model has the weights and biases from the pre-trained GPT-2 model

# Chatbot Interface using the loaded GPT-2 model
tokenizer = GPT2Tokenizer.from_pretrained(pretrained_model_name)

In [ ]:
def chatbot_prompt(prompt, model):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")

    # Generate text
    output = model.generate(input_ids, max_length=100, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95, temperature=0.7)

    # Decode and return the generated text
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text


# Test the chatbot interface
user_input = input("User: ")
while user_input.lower() != 'exit':
    chatbot_response = chatbot_prompt(user_input, empty_model)
    print(f"Chatbot: {chatbot_response}")
    user_input = input("User: ")

In [ ]:
# Unload GPT-2 model weights
gpt2_model.reset_parameters()

In [ ]:
# Download a sample DALL-E image
image_url = "https://storage.googleapis.com/sfr-vision-language-research/CLIP/demo.jpg"
image = Image.open(requests.get(image_url, stream=True).raw)

# Load CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, clip_transform = clip.load("ViT-B/32", device=device)

In [ ]:



# Function to generate text based on GPT-2 model
def generate_text(prompt, model, tokenizer):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    output = model.generate(input_ids, max_length=100, num_return_sequences=1, no_repeat_ngram_size=2)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text

# Function to generate image based on CLIP model
def generate_image(prompt, model, transform):
    text = clip.tokenize([prompt]).to(device)
    image = transform(image).unsqueeze(0).to(device)
    similarity = (model.encode_image(image) @ model.encode_text(text)).item()
    return similarity

# Toggle between the models
use_gpt2 = True  # Set to True if using GPT-2, False if using the image generation model

if use_gpt2:
    # Use GPT-2 model
    print("Using GPT-2 model")
    gpt2_prompt = "A conversation about technology and AI."
    generated_text = generate_text(gpt2_prompt, gpt2_model, gpt2_tokenizer)
    print("Generated Text:")
    print(generated_text)

else:
    # Use DALL-E image generation model
    print("Using CLIP and DALL-E image generation model")
    clip_prompt = "A surreal image with dreamy colors."
    similarity = generate_image(clip_prompt, clip_model, clip_transform)
    print("Similarity Score:", similarity)
    image.show()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# Define a simple dataset
class CustomDataset(Dataset):
    def __init__(self, data_size=50, input_size=5, output_size=10):
        self.data = torch.randn(data_size, input_size)
        self.labels_text_gen = torch.randn(data_size, output_size)
        self.labels_image_gen = torch.randn(data_size, output_size)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels_text_gen[idx], self.labels_image_gen[idx]

# Define the model
class CustomModel(nn.Module):
    def __init__(self, input_size, text_gen_size, image_gen_size):
        super(CustomModel, self).__init__()

        # Nodes for different tasks
        self.text_gen_node = nn.Linear(input_size, text_gen_size)
        self.image_gen_node = nn.Linear(input_size, image_gen_size)

    def forward(self, input_data, use_text_gen_node):
        if use_text_gen_node:
            return self.text_gen_node(input_data)
        else:
            return self.image_gen_node(input_data)

# Hyperparameters
input_size = 5
text_gen_size = 10
image_gen_size = 10
learning_rate = 0.001
num_epochs = 20

# Create dataset and dataloader
dataset = CustomDataset(data_size=50, input_size=input_size, output_size=max(text_gen_size, image_gen_size))
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# Initialize the model
model = CustomModel(input_size, text_gen_size, image_gen_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    for batch_data, batch_labels_text_gen, batch_labels_image_gen in dataloader:
        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        use_text_gen_node = torch.rand(1) > 0.5  # Randomly decide whether to use text_gen_node or image_gen_node
        output = model(batch_data, use_text_gen_node)

        # Compute the loss
        if use_text_gen_node:
            loss = criterion(output, batch_labels_text_gen)
        else:
            loss = criterion(output, batch_labels_image_gen)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

    # Print the loss for this epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Example usage
input_data_example = torch.randn(1, input_size)
output_example = model(input_data_example, use_text_gen_node=True)
print("Output for text generation node:", output_example.detach().numpy())


In [ ]:
# Install required libraries
!pip install torch torchvision transformers tiktoken

In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from torchvision import transforms
from PIL import Image
import tiktoken

In [ ]:

# Load GPT-2 model and tokenizer
model_name = "gpt2"  # You can use other GPT-2 variants like "gpt2-medium", "gpt2-large", etc.
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Example text for GPT-2
prompt_text = "Once upon a time in a"

# Tokenize input text
input_ids = tokenizer.encode(prompt_text, return_tensors="pt")

# Generate text
output = model.generate(input_ids, max_length=100, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95, temperature=0.7)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print("Generated Text:")
print(generated_text)

# Example image generation using torchvision
# Note: This is a simple example and may not produce meaningful images
transform = transforms.Compose([transforms.Resize((256, 256)), transforms.ToTensor()])
fake_image_data = torch.rand((1, 3, 256, 256))  # Replace this with your actual image data
fake_image = transforms.functional.to_pil_image(fake_image_data[0])
fake_image.show()


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Model, GPT2Config


# Define a simple dataset with random data
class CustomDataset(Dataset):
    def __init__(self, num_samples=100):
        self.data = torch.randn(num_samples, 10)  # Replace 10 with appropriate feature size

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Define the custom model
class CustomModel(nn.Module):
    def __init__(self, input_size, text_gen_size, image_gen_size, self_training_size, manual_training_size):
        super(CustomModel, self).__init__()

        # Nodes for different tasks
        self.text_gen_node = nn.Linear(in_features=input_size, out_features=text_gen_size)
        self.image_gen_node = nn.Linear(in_features=input_size, out_features=image_gen_size)
        self.self_training_node = nn.Linear(in_features=input_size, out_features=self_training_size)
        self.manual_training_node = nn.Linear(in_features=input_size, out_features=manual_training_size)

    def forward(self, input_data, is_self_training, is_manual_training):
        # Forward pass through the nodes based on conditions

        if is_self_training:
            self_training_output = self.self_training_node(input_data)
            return self_training_output

        if is_manual_training:
            manual_training_output = self.manual_training_node(input_data)
            return manual_training_output

        # Assuming input_data has appropriate features for text or image generation
        # For simplicity, let's assume input_data is a tensor for text generation
        text_gen_output = self.text_gen_node(input_data)
        image_gen_output = self.image_gen_node(input_data)

        # Return both text and image generation outputs
        return text_gen_output, image_gen_output

# Instantiate the model
input_size = 10
text_gen_size = 10
image_gen_size = 10
self_training_size = 10
manual_training_size = 10

custom_model = CustomModel(input_size, text_gen_size, image_gen_size, self_training_size, manual_training_size)

# Instantiate the dataset and dataloader
dataset = CustomDataset(num_samples=50)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

# Define optimizer and criterion
optimizer = torch.optim.Adam(custom_model.parameters(), lr=0.001)
criterion = nn.MSELoss()

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for batch_data in dataloader:
        # Randomly choose whether to self-train or manually train
        is_self_training = torch.rand(1) > 0.5
        is_manual_training = ~is_self_training

        # Forward pass
        output = custom_model(batch_data, is_self_training.item(), is_manual_training.item())

        # Dummy loss (replace with actual loss computation based on your task)
        loss = criterion(output, torch.randn_like(output))

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

# Generate some random input data for testing
test_input = torch.randn(1, input_size)

# Test text generation
text_gen_output, _ = custom_model(test_input, is_self_training=False, is_manual_training=False)
print(f'Text Generation Output: {text_gen_output}')

# Test self-training
self_training_output = custom_model(test_input, is_self_training=True, is_manual_training=False)
print(f'Self-Training Output: {self_training_output}')

# Test manual training
manual_training_output = custom_model(test_input, is_self_training=False, is_manual_training=True)
print(f'Manual Training Output: {manual_training_output}')


Epoch [1/10], Loss: 1.1144124269485474
Epoch [1/10], Loss: 1.8603613376617432
Epoch [1/10], Loss: 1.5061668157577515
Epoch [1/10], Loss: 1.3230456113815308
Epoch [1/10], Loss: 1.1251544952392578
Epoch [1/10], Loss: 1.4470881223678589
Epoch [1/10], Loss: 1.4572839736938477
Epoch [2/10], Loss: 1.5383944511413574
Epoch [2/10], Loss: 1.4510440826416016
Epoch [2/10], Loss: 1.7896969318389893
Epoch [2/10], Loss: 1.527132511138916
Epoch [2/10], Loss: 1.8367656469345093
Epoch [2/10], Loss: 1.0674484968185425
Epoch [2/10], Loss: 1.7676489353179932
Epoch [3/10], Loss: 1.6430288553237915
Epoch [3/10], Loss: 1.5699056386947632
Epoch [3/10], Loss: 1.209604024887085
Epoch [3/10], Loss: 1.316046953201294
Epoch [3/10], Loss: 1.7769778966903687
Epoch [3/10], Loss: 1.5164004564285278
Epoch [3/10], Loss: 1.0896966457366943
Epoch [4/10], Loss: 1.5018396377563477
Epoch [4/10], Loss: 1.3668925762176514
Epoch [4/10], Loss: 1.4894083738327026
Epoch [4/10], Loss: 0.9686957597732544
Epoch [4/10], Loss: 1.239022

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2Model
#from tiktoken import Tiktoken

# Load a pre-trained GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt2_model = GPT2Model.from_pretrained("gpt2")

# Add a new pad token
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token = '[PAD]'

# Define a custom dataset class
class CustomDataset(Dataset):
    def __init__(self, texts):
        self.texts = texts

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx]

# Define a custom model class
class CustomModel(nn.Module):
    def __init__(self, gpt2_model):
        super(CustomModel, self).__init__()
        self.gpt2_model = gpt2_model

        # Additional nodes for different tasks can be added here

    def forward(self, input_data):
        # Assuming input_data is a tensor of tokenized text
        gpt2_output = self.gpt2_model(input_data).last_hidden_state
        return gpt2_output

# Load a small dataset for demonstration purposes
texts = [
    "This is the first example sentence.",
    "Here's another example for testing.",
    "We need a bit more data for training.",
    "A small dataset, but good for a demo.",
]

# Tokenize the texts using GPT-2 tokenizer
tokenized_texts = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)

# Instantiate the custom dataset and dataloader
dataset = CustomDataset(tokenized_texts["input_ids"])
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# Instantiate the custom model
custom_model = CustomModel(gpt2_model)

# Define optimizer and criterion
optimizer = torch.optim.Adam(custom_model.parameters(), lr=0.001)
criterion = nn.MSELoss()

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    for batch_data in dataloader:
        # Assuming input_data is a tensor of tokenized text
        input_data = batch_data["input_ids"]

        # Forward pass
        output = custom_model(input_data)

        # Dummy loss (replace with actual loss computation based on your task)
        loss = criterion(output, torch.randn_like(output))

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

# Generate some random input data for testing
test_text = "A test sentence for evaluation."

# Tokenize the test text
tokenized_test_text = tokenizer(test_text, return_tensors="pt", padding=True, truncation=True)

# Test the custom model
test_output = custom_model(tokenized_test_text["input_ids"])
print(f'Test Output: {test_output}')


IndexError: ignored

In [ ]:
output = custom_model(batch_data["input_ids"])


IndexError: ignored